 # Assignment 5

In [2]:
import pandas as pd # Do not change these imports
import numpy as np
import math

### Part A

In [3]:
data = {"Number of Shares" : 250000,
        "Price Per Share" : 60,
        "Debt" : 310000,
        "Excess Cash" : 1000000,
        "FCF" : 500000,
        "High Growth Rate" : 0.1,
        "rwacc": 0.09,
        "re" : 0.11}
data

{'Number of Shares': 250000,
 'Price Per Share': 60,
 'Debt': 310000,
 'Excess Cash': 1000000,
 'FCF': 500000,
 'High Growth Rate': 0.1,
 'rwacc': 0.09,
 're': 0.11}

IDX is a privately held firm. You are considering making an offer to acquire the equity
of IDX. You have been informed that the founder of IDX is willing to sell all of the firm’s
shares (`Number of Shares`) for a price of `Price Per Share` per share. You are evaluating whether this is a fair price.

IDX currently has debt of `Debt` and excess cash of `Excess Cash`. You have estimated that
the firm will have free cash flows of `FCF` one year from today, and that this amount will
grow by `High Growth Rate` per year over the following three years (i.e. during years 2-4). You have also
calculated that IDX has a weighted average cost of capital of `rwacc` and a cost of equity capital
of `re`. In addition, you expect the future free cash flows for IDX beyond year 4 to grow at
a constant long run annual growth rate. What long run annual growth rate in free cash flows
would be consistent with a share price of `Price Per Share` today?

Write the function ```growth_rate```, which calculates the long run annual growth rate in the situation described above. This function takes in a dictionary `data`, formatted as above.

In [4]:
def growth_rate(data):
    hgh = data['High Growth Rate']
    rwacc = data['rwacc']
    pvfcf_year_I_IV = (1 - ((1 + hgh) / (1 + rwacc)) ** 4) * data['FCF'] / (rwacc - hgh)
    consistent_ev = data['Price Per Share'] * data['Number of Shares'] + data['Debt'] - data['Excess Cash']
    long_run = consistent_ev - pvfcf_year_I_IV
    long_run_norm = (1 + rwacc) ** 4 * long_run / ((1 + hgh) ** 3 * data['FCF'])
    ret = (rwacc * long_run_norm - 1) / (1 + long_run_norm)
    
    return ret

growth_rate(data)

0.05022904006781029

### Part B

We first need to clean up the data. For the sake of this assignment, you are provided the function ```aggregate```. However, data cleaning is the first step of any financial data analytics and thus you need to conceptually understand what is written in the function below. 

The function aggregates the supplied data to annual frequency. This function will take a  string that represents a file path to a csv file formatted like the one supplied. ```aggregate``` will output a DataFrame formatted like the one supplied. 

First, it removes all rows where ```PRC``` is 0, and where ```DIVAMT``` is 0. It is fine if after removing these rows, your aggregated DataFrame "skips" certain years.

Then, for each unique year in each CUSIP, the function adds a row to the ```output``` DataFrame. The ```CUSIP``` entry represents the CUSIP of the security, the ```Year``` entry represents which year was aggregated, the ```Dividends``` entry represents the total amount of dividends paid during the year, and the ```Price``` column represents the price of the security at the last available month of the year.

Note that the ```Dividends``` column only contains dividends paid during that year. For example, if the first available month of a certain security happens to be November 1990, then the 1990 row will only contain dividends paid in November and December of 1990.
    
Finally, the function removes all securities where there is only one year's data for dividends paid. 



In [5]:
df = pd.read_csv("monthlycrsp.csv")
df


PERMNO        date                     COMNAM     CUSIP  DIVAMT  \
0         10000  1985-12-31                        NaN  68391610     NaN   
1         10000  1986-01-31  OPTIMUM MANUFACTURING INC  68391610     NaN   
2         10000  1986-02-28  OPTIMUM MANUFACTURING INC  68391610     NaN   
3         10000  1986-03-31  OPTIMUM MANUFACTURING INC  68391610     NaN   
4         10000  1986-04-30  OPTIMUM MANUFACTURING INC  68391610     NaN   
...         ...         ...                        ...       ...     ...   
4384943   93436  2022-08-31                  TESLA INC  88160R10     0.0   
4384944   93436  2022-09-30                  TESLA INC  88160R10     NaN   
4384945   93436  2022-10-31                  TESLA INC  88160R10     NaN   
4384946   93436  2022-11-30                  TESLA INC  88160R10     NaN   
4384947   93436  2022-12-30                  TESLA INC  88160R10     NaN   

               PRC     SHROUT  
0              NaN        NaN  
1         -4.37500     3680.0  
2         -3.25000     3680.0  
3         -4.43750     3680.0  
4         -4.00000     3793.0  
...            ...        ...  
4384943  275.60999  3133470.0  
4384944  265.25000  3158000.0  
4384945  227.53999  3157752.0  
4384946  194.70000  3157752.0  
4384947  123.18000  3157752.0  

[4384948 rows x 7 columns]

In [6]:
def aggregate(path):
    df = pd.read_csv(path) # Do not change this line of code
    
    # Drop Unneccesary columns
    df.drop(['PERMNO', 'COMNAM'], axis=1, inplace = True)
    
    # Replace all NaN values with 0
    df = df.fillna(0)
    
    # Negative is used to represent something else in the data, so abs will remove the negative
    df['PRC'] = abs(df['PRC'])
    
    # Filter out rows where price is zero or DIVAMT is zero
    df = df.loc[df["PRC"] != 0]
    df = df.loc[df["DIVAMT"] != 0]
    
    # Replace date
    df["date"] = df["date"].str.slice(stop = 4)
    
    groups = df.groupby(["CUSIP", "date"], group_keys = False)
    

    price =  groups["PRC"].agg("last")
    divs = list(groups["DIVAMT"].sum())
    cusips = list(price.index.get_level_values(0))
    year = list(price.index.get_level_values(1))
    price = list(price)
    
    
    output = pd.DataFrame(data = {
        "CUSIP" : cusips,
        "Year" : year,
        "Dividends" : divs,
        "Price" : price
    })
    
    
    # filter singletons
    groups = output.groupby(["CUSIP"], group_keys = False)
    filtered = groups.filter(lambda x: len(x) != 1).reset_index().drop(['index'], axis = 1)
    
    return filtered
aggregate("monthlycrsp.csv")

CUSIP  Year  Dividends   Price
0       00020910  1991    0.40000  39.000
1       00020910  1992    0.44000  31.875
2       00020910  1993    0.24000  24.500
3       00036020  2006    0.40000  26.280
4       00036020  2007    0.36000  19.820
...          ...   ...        ...     ...
181435  Y8977Y10  2016    1.45000  15.360
181436  Y9384M10  2014    0.15978  24.450
181437  Y9384M10  2015    1.10840  21.010
181438  Y9384M10  2016    1.26085  17.850
181439  Y9384M10  2017    1.00800  19.400

[181440 rows x 4 columns]

Now that we have aggregated the data to annual frequency, we consider which percentage of securities are overvalued by the Dividend Discount Model. Write the function ```overvalued```, which takes in a DataFrame ```annual_data``` (this is the output of ```aggregate```). With this annual data:

1)	Using annual dividends, for each security (identified by CUSIP), calculate the annual dividend growth. Even though some years may be "skipped" in the above data due to 0 dividends, **treat them as if they were consecutive years** in your calculations. For example, if you found that the security returned a dividend of 1 dollar in 1990, and 1.2 dollars in 1992, you should calculate the dividend growth in 1992 to be 20\%.

2) You can calculate the average dividend growth of each security by taking the average of each security's annual dividend growth.

3)	You can similarly calculate the equity cost of capital by calculating the annual equity cost of capital (including the dividend), and taking the average. Note that in this step and the previous, these calculations will never fail to produce a number, since we have filtered out all cases where the Dividends/Stock Price are 0, and all securities contain at least 2 entries.

4)	Use “Dividend discount model (constant growth model)” to calculate the price on that security’s last available date.

5)	Compare model-implied price with actual price.

You may find the ```groupby``` method for DataFrames helpful in this question.

In [7]:
def avg_growth(lst):
    cum = 0
    for x in range(1, len(lst)):
        cum += lst.iloc[x] / lst.iloc[x - 1] - 1
    return cum * 1.0 / (len(lst) - 1)

def avg_ecc(div, price):
    cum = 0
    for x in range(1, len(div)):
        cum += ((price.iloc[x] + div.iloc[x]) / price.iloc[x - 1]) - 1
    return cum * 1.0 / (len(div) - 1)

In [8]:
agg = aggregate("monthlycrsp.csv")

In [9]:
def overvalued(annual_data):
    over_count = 0
    groups = 0
    for group in annual_data.groupby(['CUSIP'], group_keys = False):
        g = avg_growth(group[1]["Dividends"])
        re = avg_ecc(group[1]["Dividends"], group[1]["Price"])
        latest = group[1].iloc[-1]
        if re == g:
            if latest['Dividends'] * (1 + g) != 0:
                over_count += 1
            groups += 1
            continue
        model_price = latest['Dividends'] * (1 + g) / (re - g)
        if model_price > latest['Price'] or re < g:
            over_count += 1
        groups += 1
    return over_count * 1.0 / groups 

overvalued(agg)

0.634944